In [ ]:
pip install wandb datasets transformers trl torch peft

In [ ]:
import wandb
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
from trl import SFTTrainer, SFTConfig
import torch
from peft import LoraConfig, get_peft_model

In [ ]:
# wandb 설정
wandb.init(project='Hanghae99-8-basic')
wandb.run.name = 'gpt-finetuning-with-lora'

In [ ]:
# 모델과 토크나이저 로드
model_name = "facebook/opt-350m"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# 데이터셋 로드
dataset = load_dataset("sahil2801/CodeAlpaca-20k", split="train")

print("=== 데이터셋 예시 ===")
print(dataset[0])

In [ ]:
# 프롬프트 포매팅 함수 정의
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        # input이 비어있는 경우 처리
        input_text = example['input'][i].strip() if example['input'][i] else ""
        
        # 형식화된 프롬프트 생성
        text = f"[Instruction]\n{example['instruction'][i].strip()}\n\n"
        if input_text:
            text += f"[Input]\n{input_text}\n\n"
        text += f"[Output]\n{example['output'][i].strip()}"
        
        output_texts.append(text)
    return output_texts

# 데이터 콜레이터 설정
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [ ]:
# 여러 lora_r 값으로 실험
lora_r_values = [8, 128, 256]

for lora_r in lora_r_values:
    print(f"\n=== Training with LoRA rank {lora_r} ===")
    
    # 기본 모델 로드
    model = AutoModelForCausalLM.from_pretrained(model_name)
    
    # LoRA 설정
    lora_config = LoraConfig(
        r=lora_r,
        lora_alpha=32,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )
    
    # LoRA 적용
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()
    
    # SFTTrainer 설정 및 학습
    trainer = SFTTrainer(
        model,
        train_dataset=dataset,
        args=SFTConfig(
            output_dir=f"/tmp/clm-instruction-tuning-lora-{lora_r}",
            max_seq_length=128
        ),
        formatting_func=formatting_prompts_func,
        data_collator=data_collator,
    )

    # 학습 시작
    trainer.train()
    
    # 메모리 사용량 출력
    if torch.cuda.is_available():
        print('Max Alloc:', round(torch.cuda.max_memory_allocated(0)/1024**3, 1), 'GB')
    
    # 모델 저장
    trainer.save_model()
    
    # CUDA 캐시 정리
    if torch.cuda.is_available():
        torch.cuda.empty_cache()